# GATK Variant Filtration <a class="tocSkip">
   

**January 2024**

<font size=4>This GATK tutorial will help you become familiar with using GATK tools to filter annotated variants. The notebook illustrates the following steps. 

- Use GATK to annotate a VCF with scores from an isolated forest model with Variant Extract-Train-Score (VETS) The higher the score, the more likely that the variant is real.
- Use GATK to annotate a VCF with scores from a Convolutional Neural Network (CNN)
- Generate 1D CNN models
- Apply tranche filtering to VCF based on scores from an annotation in the INFO field
- Calculate concordance metrics</font>

_This tutorial was last tested with GATK v4.4.0 and IGV v2.8.0._ See [GATK Tool Documentation](https://gatk.broadinstitute.org/hc/en-us/articles/360037224712) for further information on the tools we use below.

# Set up your Notebook

## Set cloud environment values
If you opened this notebook and didn't adjust any cloud environment values, now's the time to edit them. Click on the gear icon in the upper right to edit your Cloud Environment form. Set the values as specified below:

| Option | Value |
| ------ | ------ |
| Application Configuration | Custom Environment |
| Container Image | us.gcr.io/broad-dsde-methods/gatk-workshop-terra-jupyter-image:1.0rc11 |
| Creation Timeout Limit | 20 |
| CPUs | 4 |
| Memory | 15 GB |
| Enable Autopause | True |
| Minutes of Inactivity | 60 |
| Disk size | 100 GB |


Click the "Update" button when you are done, and Terra will begin to create a new runtime with your settings. When it is finished, it will pop up asking you to apply the new settings. In the meantime, you can continue with the setup instructions below. Note that because it is a custom image, this may take up to 10 minutes.

## Check kernel type
A kernel is a _computational engine_ that executes the code in the notebook. For this particular notebook, we will be using a custom Python 3 kernel so we can execute GATK commands using _Python Magic_ (`!`). 

To add your custom kernel to the notebook, go to the notebook's terminal, which you can find by clicking on the terminal icon at the bottom of the column to the far right of this notebook. You will be navigated to a new tab with your terminal. Type the command "setup_gatk_env" in the terminal, and hit enter. It will run for about 3 minutes and once it has printed that it has successfully installed, you can navigate back to this tab. You should now be able to set the Kernal from the Kernel menu in the upper left. 
Click on `Kernel` > `Change kernel` > `Python[conda env:gatkconda]`
If you do not see this new custom option close and restart your Jupyter notebook

In the upper right corner of the notebook, just under the Notebook Runtime, it should now say `Python[conda env:gatkconda]`. 

If you found the custom image and kernel instructions utterly confusing, navigate to a [more complete set of instructions with screenshots.](https://github.com/broadinstitute/gatk-workshop-terra-jupyter-image/wiki/Using-the-gatk%E2%80%90workshop%E2%80%90terra%E2%80%90jupyter%E2%80%90image-in-the-Terra-Jupyter-environment)

## Set up your files
Your notebook has a temporary folder that exists so long as your cluster is running. To see what files are in your notebook environment at any time, you can click on the Jupyter logo in the upper left corner. 

For this tutorial, we need to copy some files from this temporary folder to and from our workspace bucket. Run the two commands below to set up the workspace bucket variable and the file paths inside your notebook.

<font color = "green"> **Tool Tip:** To run a cell in a notebook, press `SHIFT + ENTER`</font>

In [1]:
# Set your workspace bucket variable for this notebook.
import os
# BUCKET = os.environ['WORKSPACE_BUCKET']

In [2]:
# Set workshop variable to access the most recent materials
WORKSHOP = "workshop_2002"

In [3]:
# Create a path we can reference for the rest of the notebook:
RESOURCE_DIR = f"gs://broad-dsde-methods-public/hg19"
BASE_DATA_DIR = f"gs://gatk-tutorials/{WORKSHOP}"
LOCAL_DATA_DIR = "/home/jupyter/notebooks"

In [4]:
# Create directories for your files to live inside this notebook
! mkdir -p $LOCAL_DATA_DIR/Output

## Check data permissions
For this tutorial, we have hosted the starting files in a public Google bucket. We will first check that the data is available to your user account, and if it is not, we simply need to install Google Cloud Storage.

In [5]:
# Check if data is accessible. The command should list several gs:// URLs.
! gsutil ls $BASE_DATA_DIR/2-germline/
! gsutil ls $RESOURCE_DIR

gs://gatk-tutorials/workshop_2002/2-germline/trio.ped
gs://gatk-tutorials/workshop_2002/2-germline/CNNScoreVariants/
gs://gatk-tutorials/workshop_2002/2-germline/bams/
gs://gatk-tutorials/workshop_2002/2-germline/gvcfs/
gs://gatk-tutorials/workshop_2002/2-germline/illumina_platinum/
gs://gatk-tutorials/workshop_2002/2-germline/intervals/
gs://gatk-tutorials/workshop_2002/2-germline/ref/
gs://gatk-tutorials/workshop_2002/2-germline/resources/
gs://broad-dsde-methods-public/hg19/
gs://broad-dsde-methods-public/hg19/1000G_omni2.5.b37.chr20.vcf.gz
gs://broad-dsde-methods-public/hg19/1000G_omni2.5.b37.chr20.vcf.gz.tbi
gs://broad-dsde-methods-public/hg19/1000G_phase1.snps.high_confidence.b37.chr20.vcf.gz
gs://broad-dsde-methods-public/hg19/1000G_phase1.snps.high_confidence.b37.chr20.vcf.gz.tbi
gs://broad-dsde-methods-public/hg19/Axiom_Exome_Plus.genotypes.all_populations.poly.chr20.vcf.gz
gs://broad-dsde-methods-public/hg19/Axiom_Exome_Plus.genotypes.all_populations.poly.chr20.vcf.gz.tbi
gs:

In [8]:
# If you do not see gs:// URLs listed above, run this cell to install Google Cloud Storage. 
# Afterwards, restart the kernel with Kernel > Restart.
#! pip install google-cloud-storage

## Download Data to the Notebook 
Some tools are not able to read directly from a Google bucket, so we download their files to our local notebook folder.

In [9]:
# Start by downloading the base data:
! time gsutil -m cp -r $BASE_DATA_DIR/2-germline $LOCAL_DATA_DIR/

print()

# Now download the other reference info:
! time gsutil -m cp $RESOURCE_DIR/*.vcf.gz* $LOCAL_DATA_DIR/2-germline/resources

/usr/bin/sh: 1: time: not found

/usr/bin/sh: 1: time: not found


In [10]:
# Let's alias our variant file so we can just use it later:
VCF_FILE = f"{BASE_DATA_DIR}/2-germline/CNNScoreVariants/vcfs/g94982_b37_chr20_1m_15871.vcf.gz"

# Run the Variant Extract-Train-Score (VETS) on the VCF

The Variant Extract-Train-Score (VETS) toolchain uses an isolation-forest model to flag probable artifacts.

At a high-level, using this tool is a 3-step process:

1. Extract the Variant Annotations
2. Train the Variant Annotations Model
3. Score the Variants

This requires that you already have a set of variants that you have somehow validated which you can use to train the models.

Note that because INDELs and SNPs have different inherent properties, they each require their own model.  This is also reflected in the known-variant datasets that are used to train the models. VETS creates both models at the same time as its default behavior and is used that way in this demo.

In this example the variant calls to be filtered as well as the "truth" data are provided.

For more information, see the [GATK Release Notes on VETS](https://github.com/broadinstitute/gatk/blob/ah_var_store/scripts/variantstore/docs/release_notes/VETS_Release.pdf).

## Extract the Variant Annotations

Let's start by extracting the variants using the `ExtractVariantAnnotations`.

Here we are using truth data from `hapmap`, `omni`, `1000 genomes`, `mills`, and `axiomPoly` to help train the models.  For this exercise the truth resource data has been truncated to only the region represented in our example data.

The first three are best for SNP model training and the second two are best for INDEL model training.

In [11]:
! gatk ExtractVariantAnnotations \
    -mode SNP \
    -mode INDEL \
    -V $VCF_FILE \
    -O $LOCAL_DATA_DIR/Output/output.extracted \
    -A QD \
    -A FS \
    -A SOR \
    -A MQRankSum \
    -A ReadPosRankSum \
    --resource:hapmap,training=true,calibration=true $LOCAL_DATA_DIR/2-germline/resources/hapmap_3.3.b37.chr20.vcf.gz \
    --resource:omni,training=true,calibration=true $LOCAL_DATA_DIR/2-germline/resources/1000G_omni2.5.b37.chr20.vcf.gz \
    --resource:1000G,training=true $LOCAL_DATA_DIR/2-germline/resources/1000G_phase1.snps.high_confidence.b37.chr20.vcf.gz \
    --resource:mills,training=true,calibration=true $LOCAL_DATA_DIR/2-germline/resources/Mills_and_1000G_gold_standard.indels.b37.chr20.vcf.gz \
    --resource:axiomPoly,training=true,calibration=true $LOCAL_DATA_DIR/2-germline/resources/Axiom_Exome_Plus.genotypes.all_populations.poly.chr20.vcf.gz


Using GATK jar /gatk/gatk-package-4.5.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /gatk/gatk-package-4.5.0.0-local.jar ExtractVariantAnnotations -mode SNP -mode INDEL -V gs://gatk-tutorials/workshop_2002/2-germline/CNNScoreVariants/vcfs/g94982_b37_chr20_1m_15871.vcf.gz -O /home/jupyter/notebooks/Output/output.extracted -A QD -A FS -A SOR -A MQRankSum -A ReadPosRankSum --resource:hapmap,training=true,calibration=true /home/jupyter/notebooks/2-germline/resources/hapmap_3.3.b37.chr20.vcf.gz --resource:omni,training=true,calibration=true /home/jupyter/notebooks/2-germline/resources/1000G_omni2.5.b37.chr20.vcf.gz --resource:1000G,training=true /home/jupyter/notebooks/2-germline/resources/1000G_phase1.snps.high_confidence.b37.chr20.vcf.gz --resource:mills,training=true,calibration=true /home/jupyter/notebooks/2-germline/resources/Mills_and_1000G_

## Train the Variant Annotations

Now we can use the variants to train our model using the `TrainVariantAnnotationsModel`.

In [9]:
! gatk TrainVariantAnnotationsModel \
   --annotations-hdf5 $LOCAL_DATA_DIR/Output/output.extracted.annot.hdf5 \
   -mode SNP \
   -mode INDEL \
   -O $LOCAL_DATA_DIR/Output/output.trained


Using GATK jar /gatk/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /gatk/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar TrainVariantAnnotationsModel --annotations-hdf5 /home/jupyter/notebooks/Output/output.extracted.annot.hdf5 -mode SNP -mode INDEL -O /home/jupyter/notebooks/Output/output.trained
16:39:50.243 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/gatk/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
16:39:50.292 INFO  TrainVariantAnnotationsModel - ------------------------------------------------------------
16:39:50.296 INFO  TrainVariantAnnotationsModel - The Genome Analysis Toolkit (GATK) v4.4.0.0
16:39:50.296 INFO  TrainVariantAnnotationsModel - For support and documentation go to https://software.broadinstitute.org/gatk/
16:39:50.297 INFO

## Finally, we will score the Variant Annotations

Now we can score the variants in our model using the `ScoreVariantAnnotations`.

Note that the filtering threshold has been set:
  --snp-calibration-sensitivity-threshold 0.997
  --indel-calibration-sensitivity-threshold 0.997

Make sure that the resources used here match those from the extraaction step above.

In [12]:
! gatk ScoreVariantAnnotations \
    -mode SNP \
    -mode INDEL \
    -V $VCF_FILE \
    -O $LOCAL_DATA_DIR/Output/output.score \
    --model-prefix $LOCAL_DATA_DIR/Output/output.trained \
    -A QD \
    -A FS \
    -A SOR \
    -A MQRankSum \
    -A ReadPosRankSum \
    --resource:hapmap,training=true,calibration=true $LOCAL_DATA_DIR/2-germline/resources/hapmap_3.3.b37.chr20.vcf.gz \
    --resource:omni,training=true,calibration=true $LOCAL_DATA_DIR/2-germline/resources/1000G_omni2.5.b37.chr20.vcf.gz \
    --resource:1000G,training=true $LOCAL_DATA_DIR/2-germline/resources/1000G_phase1.snps.high_confidence.b37.chr20.vcf.gz \
    --resource:mills,training=true,calibration=true $LOCAL_DATA_DIR/2-germline/resources/Mills_and_1000G_gold_standard.indels.b37.chr20.vcf.gz \
    --resource:axiomPoly,training=true,calibration=true $LOCAL_DATA_DIR/2-germline/resources/Axiom_Exome_Plus.genotypes.all_populations.poly.chr20.vcf.gz \
    --resource:extracted,extracted=true $LOCAL_DATA_DIR/Output/output.extracted.vcf.gz \
    --snp-calibration-sensitivity-threshold 0.997 \
    --indel-calibration-sensitivity-threshold 0.997



Using GATK jar /gatk/gatk-package-4.5.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /gatk/gatk-package-4.5.0.0-local.jar ScoreVariantAnnotations -mode SNP -mode INDEL -V gs://gatk-tutorials/workshop_2002/2-germline/CNNScoreVariants/vcfs/g94982_b37_chr20_1m_15871.vcf.gz -O /home/jupyter/notebooks/Output/output.score --model-prefix /home/jupyter/notebooks/Output/output.trained -A QD -A FS -A SOR -A MQRankSum -A ReadPosRankSum --resource:hapmap,training=true,calibration=true /home/jupyter/notebooks/2-germline/resources/hapmap_3.3.b37.chr20.vcf.gz --resource:omni,training=true,calibration=true /home/jupyter/notebooks/2-germline/resources/1000G_omni2.5.b37.chr20.vcf.gz --resource:1000G,training=true /home/jupyter/notebooks/2-germline/resources/1000G_phase1.snps.high_confidence.b37.chr20.vcf.gz --resource:mills,training=true,calibration=true /home/

In [ ]:
! cat $LOCAL_DATA_DIR/Output/output.extracted.annot.hdf5
TODO:
import h5py    
import numpy as np    
f1 = h5py.File(file_name,'r+')   

## Evaluate the Unfiltered VCF

In [13]:
!gatk Concordance \
-truth $BASE_DATA_DIR/2-germline/CNNScoreVariants/vcfs/hg001_na12878_b37_truth.vcf.gz \
-eval $VCF_FILE \
-L 20:1000000-9467292 \
-S $LOCAL_DATA_DIR/Output/unfiltered_concordance.txt

Using GATK jar /gatk/gatk-package-4.5.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /gatk/gatk-package-4.5.0.0-local.jar Concordance -truth gs://gatk-tutorials/workshop_2002/2-germline/CNNScoreVariants/vcfs/hg001_na12878_b37_truth.vcf.gz -eval gs://gatk-tutorials/workshop_2002/2-germline/CNNScoreVariants/vcfs/g94982_b37_chr20_1m_15871.vcf.gz -L 20:1000000-9467292 -S /home/jupyter/notebooks/Output/unfiltered_concordance.txt
05:04:15.986 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/gatk/gatk-package-4.5.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
05:04:16.169 INFO  Concordance - ------------------------------------------------------------
05:04:16.173 INFO  Concordance - The Genome Analysis Toolkit (GATK) v4.5.0.0
05:04:16.173 INFO  Concordance - For support and documentation go to https://software.broa

## Evaluate VETS

In [14]:
!gatk Concordance \
-truth $BASE_DATA_DIR/2-germline/CNNScoreVariants/vcfs/hg001_na12878_b37_truth.vcf.gz \
-eval $LOCAL_DATA_DIR/Output/output.score.vcf.gz \
-L 20:1000000-9467292 \
-S $LOCAL_DATA_DIR/Output/vets_concordance.txt

Using GATK jar /gatk/gatk-package-4.5.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /gatk/gatk-package-4.5.0.0-local.jar Concordance -truth gs://gatk-tutorials/workshop_2002/2-germline/CNNScoreVariants/vcfs/hg001_na12878_b37_truth.vcf.gz -eval /home/jupyter/notebooks/Output/output.score.vcf.gz -L 20:1000000-9467292 -S /home/jupyter/notebooks/Output/vets_concordance.txt
05:05:23.437 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/gatk/gatk-package-4.5.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
05:05:23.606 INFO  Concordance - ------------------------------------------------------------
05:05:23.609 INFO  Concordance - The Genome Analysis Toolkit (GATK) v4.5.0.0
05:05:23.609 INFO  Concordance - For support and documentation go to https://software.broadinstitute.org/gatk/
05:05:23.609 INFO  Concordance - E

**Now look at how precision goes up (and sensitivity goes down) as we filter.**

In [17]:
# Define a quick tool to generate an F1 score from the concordance report <collapsed for ease of reading>

def print_f1_scores(concordance_file):
    with open(concordance_file, 'r') as f:
        # Skip header:
        f.readline()
        # Get SNP scores:
        _, tp, fp, fn, _, _ = f.readline().split("\t")
        snp_f1 = (2*int(tp))/((2*int(tp))+int(fp)+int(fn))
        # Get INDEL scores:
        _, tp, fp, fn, _, _ = f.readline().split("\t")
        indel_f1 = (2*int(tp))/((2*int(tp))+int(fp)+int(fn))
    ff=".5f"
    print(f"SNP F1:\t\t{snp_f1:{ff}}\nINDEL F1:\t{indel_f1:{ff}}")

In [18]:
!cat $LOCAL_DATA_DIR/Output/unfiltered_concordance.txt
print()
print_f1_scores(f"{LOCAL_DATA_DIR}/Output/unfiltered_concordance.txt")

type	TP	FP	FN	RECALL	PRECISION
SNP	10715	1218	558	0.951	0.898
INDEL	1615	1024	111	0.936	0.612

SNP F1:		0.92347
INDEL F1:	0.73998


In [19]:
!cat $LOCAL_DATA_DIR/Output/vets_concordance.txt
print()
print_f1_scores(f"{LOCAL_DATA_DIR}/Output/vets_concordance.txt")

cat: /home/jupyter/notebooks/Output/vets_concordance.txt: No such file or directory



FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/notebooks/Output/vets_concordance.txt'

---
# Run the default 1D model on the VCF with CNNScoreVariants

CNNScoreVariant is a pre-trained Convolutional Neural Network tool to score variants. This tool uses machine learning to differentiate between good variants and artifacts of the sequencing process, a fairly new approach that is especially effective at correctly calling indels. 

VQSR and Hard-filtering only take into account variant annotations. However, CNNScoreVariants 1D Model evaluates **annotations** AND **reference files**, plus or minus 64 bases from the variant. For example, it accounts for regions in the ref file that are difficult to sequence.

To enable the models to accurately filter and score variants from VCF files, we **trained** on validated VCFs (from truth models including **SynDip, Genomes in a bottle, and Platinum Genomes**) with unvalidated VCFs aligned to different reference builds (**HG19, HG38**), sequenced on **different machines**, using **different protocols**. 

In [19]:
!gatk CNNScoreVariants \
-V $VCF_FILE \
-O $LOCAL_DATA_DIR/Output/my_1d_cnn_scored.vcf \
-R gs://gcp-public-data--broad-references/hg19/v0/Homo_sapiens_assembly19.fasta

Using GATK jar /gatk/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /gatk/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar CNNScoreVariants -V gs://gatk-tutorials/workshop_2002/2-germline/CNNScoreVariants/vcfs/g94982_b37_chr20_1m_15871.vcf.gz -O /home/jupyter/notebooks/Output/my_1d_cnn_scored.vcf -R gs://gcp-public-data--broad-references/hg19/v0/Homo_sapiens_assembly19.fasta
22:02:26.982 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/gatk/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
22:02:27.030 INFO  CNNScoreVariants - ------------------------------------------------------------
22:02:27.034 INFO  CNNScoreVariants - The Genome Analysis Toolkit (GATK) v4.4.0.0
22:02:27.034 INFO  CNNScoreVariants - For support and documentation go to https://software.b

The output VCF `my_1d_cnn_scored.vcf` will now have an INFO  field CNN_1D which corresponds to the score assigned by 1D model.

In [20]:
!cat $LOCAL_DATA_DIR/Output/my_1d_cnn_scored.vcf | grep -v '##' | head -5

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	NA12878
20	1000072	rs6056638	A	G	998.77	PASS	AC=2;AF=1.00;AN=2;CNN_1D=3.256;DB;DP=32;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;POSITIVE_TRAIN_SITE;QD=31.21;SOR=0.818;VQSLOD=20.79;culprit=MQ	GT:AD:DP:GQ:PL	1/1:0,32:32:96:1027,96,0
20	1000152	rs6056639	C	T	678.77	PASS	AC=2;AF=1.00;AN=2;CNN_1D=2.313;DB;DP=28;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;POSITIVE_TRAIN_SITE;QD=24.24;SOR=0.693;VQSLOD=18.18;culprit=QD	GT:AD:DP:GQ:PL	1/1:0,28:28:81:707,81,0
20	1000179	rs6056640	A	C	532.77	PASS	AC=2;AF=1.00;AN=2;CNN_1D=3.157;DB;DP=20;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;POSITIVE_TRAIN_SITE;QD=29.60;SOR=0.914;VQSLOD=21.14;culprit=MQ	GT:AD:DP:GQ:PL	1/1:0,18:18:54:561,54,0
20	1000303	rs6056641	T	C	325.77	PASS	AC=2;AF=1.00;AN=2;CNN_1D=2.057;DB;DP=12;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;POSITIVE_TRAIN_SITE;QD=27.15;SOR=1.445;VQSLOD=17.39;culprit=QD	GT:AD:DP:GQ:PL	1/1:0,12:12:36:354,36,0
grep: write e

## Apply filters to the VCF based on the CNN_1D score with the FilterVariantTranches tool

After scoring, you can filter your VCF by applying a sensitivity threshold with the tool FilterVariantTranches. 

Note that here the threshold is set to 99.7

In [21]:
!gatk FilterVariantTranches \
-V $LOCAL_DATA_DIR/Output/my_1d_cnn_scored.vcf \
--resource gs://gcp-public-data--broad-references/hg19/v0/1000G_omni2.5.b37.vcf.gz \
--resource gs://gcp-public-data--broad-references/hg19/v0/hapmap_3.3.b37.vcf.gz \
--info-key CNN_1D \
--snp-tranche 99.7 \
--indel-tranche 99.7 \
-O $LOCAL_DATA_DIR/Output/my_1d_filtered.vcf \
--invalidate-previous-filters 

Using GATK jar /gatk/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /gatk/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar FilterVariantTranches -V /home/jupyter/notebooks/Output/my_1d_cnn_scored.vcf --resource gs://gcp-public-data--broad-references/hg19/v0/1000G_omni2.5.b37.vcf.gz --resource gs://gcp-public-data--broad-references/hg19/v0/hapmap_3.3.b37.vcf.gz --info-key CNN_1D --snp-tranche 99.9 --indel-tranche 99.9 -O /home/jupyter/notebooks/Output/my_1d_filtered.vcf --invalidate-previous-filters
22:03:11.315 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/gatk/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
22:03:11.372 INFO  FilterVariantTranches - ------------------------------------------------------------
22:03:11.376 INFO  FilterVariantTranches - 

**Now you have a neural network filtered VCF!**

## Evaluate the 1D Model 

In [22]:
!gatk Concordance \
-truth $BASE_DATA_DIR/2-germline/CNNScoreVariants/vcfs/hg001_na12878_b37_truth.vcf.gz \
-eval $LOCAL_DATA_DIR/Output/my_1d_filtered.vcf \
-L 20:1000000-9467292 \
-S $LOCAL_DATA_DIR/Output/my_1d_filtered_concordance.txt


Using GATK jar /gatk/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /gatk/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar Concordance -truth gs://gatk-tutorials/workshop_2002/2-germline/CNNScoreVariants/vcfs/hg001_na12878_b37_truth.vcf.gz -eval /home/jupyter/notebooks/Output/my_1d_filtered.vcf -L 20:1000000-9467292 -S /home/jupyter/notebooks/Output/my_1d_filtered_concordance.txt
22:14:55.244 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/gatk/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
22:14:55.297 INFO  Concordance - ------------------------------------------------------------
22:14:55.302 INFO  Concordance - The Genome Analysis Toolkit (GATK) v4.4.0.0
22:14:55.302 INFO  Concordance - For support and documentation go to https://software.broadinstit

**Now look at how precision goes up (and sensitivity goes down) as we filter.**

In [23]:
!cat $LOCAL_DATA_DIR/Output/unfiltered_concordance.txt
print()
print_f1_scores(f"{LOCAL_DATA_DIR}/Output/unfiltered_concordance.txt")

type	TP	FP	FN	RECALL	PRECISION
SNP	10715	1218	558	0.951	0.898
INDEL	1615	1024	111	0.936	0.612

SNP F1:		0.92347
INDEL F1:	0.73998


In [25]:
!cat $LOCAL_DATA_DIR/Output/my_1d_filtered_concordance.txt
print()
print_f1_scores(f"{LOCAL_DATA_DIR}/Output/my_1d_filtered_concordance.txt")


type	TP	FP	FN	RECALL	PRECISION
SNP	11204	1422	69	0.994	0.887
INDEL	1151	338	575	0.667	0.773

SNP F1:		0.93761
INDEL F1:	0.71602


**We can now compare the CNN and VETS based filtering approaches:**

In [26]:
!cat $LOCAL_DATA_DIR/Output/vets_concordance.txt
print()
print_f1_scores(f"{LOCAL_DATA_DIR}/Output/vets_concordance.txt")

type	TP	FP	FN	RECALL	PRECISION
SNP	10596	1175	677	0.94	0.9
INDEL	1608	1016	118	0.932	0.613

SNP F1:		0.91963
INDEL F1:	0.73931


For this dataset with these thresholds we can see two things:
1. The 1D-CNN is better for SNP refinement.
2. VETS is better for INDEL refinement.

Can you think of why this might be?

How could we improve the performance of the 1D-CNN?  Of VETS?